In [4]:
#%matplotlib widget  # equivalent to:
import matplotlib
import ipympl  # this automatically does a "matplotlib.use('module://ipympl.backend_nbagg')"
from matplotlib import pyplot as plt
plt.ion()
from IPython.display import Image

import numpy as np
import os
import sys
sys.path.append('/nfs/home/tobiasj/ipython/notebooks/cn_plot_style')
import cn_plot_style as cnps

sys.path.append('/srv/ipython-repo/tj_scripts/py_fluoracle')
import py_fluoracle as pyflu

import scipy as sp
import scipy.optimize

from lmfit import minimize, Parameters, fit_report
from mpl_toolkits.axes_grid1 import host_subplot, host_axes
from numpy import sin, exp
from scipy import constants
import inspect


def residual(params, func, x, data=None, eps=None):
    model = func(x, **params)

    if data is None:
        return model
    if eps is None:
        return (model - data)
    return (model - data) / eps


def create_params(func, defaults=None, verbose=True):
    args, varargs, keywords, _defaults, kwonlyargs, kwonlydefaults, annotations = inspect.getfullargspec(func)
    defaults = defaults or _defaults

    params = Parameters()
    
    if defaults is None:
        for arg in args[1:]:
            params.add(arg)
    else:
        len_defaults = len(defaults)
        for arg in args[1:-len_defaults]:
            params.add(arg)
        for arg, default in zip(args[-len_defaults:], defaults):
            params.add(arg, value=default)

    if verbose:
        print(params)
        
    return params

In [7]:
# read in all datasets in a folder
path = '/nfs/home/tobiasj/data/Spectrofluorometer/2018-06-27 - annealing with Rad52'
datasets = []

for root, dirs, files in os.walk(path):
    if root.endswith('data'):
        for file in files:
            if file.endswith('.csv'):
                filepath = os.path.join(root, file)
                dataset = pyflu.load_data(filepath)
                if dataset is not None:
                    datasets.append(dataset)

In [6]:
# Plot individual dataset
index = 1

plt.close('all')

# define the model function describing the data
# The first argument needs to be the independent variable (e.g. time)
# The following arguments are the dependent variables. A default value will be used as a start value for the fitting. 
def exponential(t, A=10000, K=-0.001, C=0):
    return A * np.exp(K * t) + C
def linear(t, m=1, n=0):
    return m*t + n

def a(f, A0, B0):
    return f * B0/A0 * (A0 - B0)
def b(A0, B0):
    return B0 / A0
def ke(A0, B0, k):
    return (A0 - B0) * k
def F(t, B0=1e-9, f=10000, k=1):
    # See Marcel Ander 2011
    A0 = 0.66/199*200/200*195*1e-9  # total volume 199 instead of 200 µl, only taken 195 µl instead of 200 µl: 0.65 nM
    #B0 = 1/394*400*1e-9  # total volume 394 instead of 400 µl: 1.02 nM
    #C = 0
    return a(f, A0, B0) / (exp(ke(A0, B0, k) * t) - b(A0, B0)) #- C) / (1 - C)

model_func = exponential
model_func = linear
model_func = F

outs = []
with cnps.cn_plot(context='paper', dark=False, usetex=False) as cnp:
    fig = plt.figure()
    ax = host_subplot(111)
    
    fig2, ax2 = plt.subplots()
    
    #ax2 = ax.twinx()
    
    lns = []
    for dataset in datasets[:2]:
        data = dataset['data'][:,1]
        Fmax = pyflu.max_fluorescence(data) * 199 / 394  # diluted 199 µl to final volume of 394 µl
        start, stop = pyflu.decay_region(data)
        x = dataset['data'][:,0][start:stop]
        y = dataset['data'][:,1][start:stop] #/ (F0 * 199 / 394)
        
        # Set first datapoint to time 0 + 30 s (~ duration needed for mixing)
        x = x - x[0] + 10
        
        # select the first seconds
        duration = 13000  # s
        x = x[:int(duration / 10)]
        y = y[:int(duration / 10)]

        # get the label for this dataset
        label = dataset['name'].replace('_', ' ').replace('1st', '').replace('2nd', '')
        #label = dataset['meta']['Labels']
        
        # get the color for this dataset
        c = cnp.color
        
        # Fit the data
        params = create_params(model_func)#, defaults=(16000, -0.0005, 2000))
        out = minimize(residual, params, args=(model_func, x, y))
        outs.append(out)
        print(fit_report(out))
        
        # Calculate and print all relevant infos from the fit
        A0 = 0.66/199*200/200*195*1e-9  # total volume 199 instead of 200 µl, only taken 195 µl instead of 200 µl: 0.65 nM
        B0i = 1/394*400*1e-9  # total volume 394 instead of 400 µl: 1.02 nM
        B0f = out.params['B0'].value #* B0
        kf = out.params['k'].value
        f = out.params['f'].value

        a_ = f * B0f / A0 * (A0 - B0f)
        b_ = B0f / A0
        ke_ = (A0 - B0f) * kf

        ##F0 = residual(outs[i].params, model_func, 0)
        ##Finf = residual(outs[i].params, model_func, np.inf)
        F0 = a_ / (1 - b_)
        Finf = - a_ / b_

        #B0e = B0i / b_  # ???
        B0e = B0f
        #k = ke_ / (A0 - B0e)  # ???
        k = kf
        t12 = np.log(b_ / (2 * b_ - 1)) / ke_

        print('Reactive ssDNA: {:.1%}'.format(B0e / B0i))
        print('F0: {:.0f} -> Finf: {:.0f}'.format(F0, Finf))
        print('Fmax: {:.0f}'.format(Fmax))
        print('kf: {:.2e}'.format(kf))
        print('ke: {:.2e}'.format(ke_))
        #print('k: {:.2e}'.format(k))
        print('t12: {:.1f}'.format(t12))
        
        # Plot the data and the fit
        lns.append(ax.plot(x[::5], (y[::5] - Finf) / (F0 - Finf), '.', label=label, color=c))
        lns.append(ax.plot(x[::5], (residual(out.params, model_func, x)[::5] - Finf) / (F0 - Finf), label='fit', color=c))
        
        # Plot the slope (reaction time) according to Marangone 2003
        def change(A0, B0, At, Bt):
            return 1 / (A0 - B0) * np.log(B0 * At / (A0 * Bt))
        
        # Maximum amount of product (i.e. dsDNA) at t=inf is min(A0, B0) = A0
        # Amount of product at t=0 is A0 * 0
        # Amount of product at t=inf is A0 * 1
        # Percent of reacted A0 is (y - Finf) / (Fmax/2 - Finf)
        Pt = A0 * (1 - (y - Finf) / (Fmax - Finf))
        ax2.plot(x[::5], change(B0i, A0, B0i - Pt, A0 - Pt)[::5])
    
    cnps.legend(*lns)

    title = 'Annealing with and without Rad52'
    x_label = dataset['meta']['Xaxis']
    #y_label = dataset['meta']['Yaxis']
    y_label = 'Reactive ssDNA (%)'

    ax.set_title(title)
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    
    ax2.set_title('Changes in reactant concentration')
    ax2.set_xlabel(x_label)
    ax2.set_ylabel('1/[Reactive ssDNA]')

    fig.show()
    
    fig2.show()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

Parameters([('B0', <Parameter 'B0', 1e-09, bounds=[-inf:inf]>), ('f', <Parameter 'f', 10000, bounds=[-inf:inf]>), ('k', <Parameter 'k', 1, bounds=[-inf:inf]>)])
[[Fit Statistics]]
    # function evals   = 894
    # data points      = 1300
    # variables        = 3
    chi-square         = 33523899.431
    reduced chi-square = 25847.262
    Akaike info crit   = 13210.944
    Bayesian info crit = 13226.455
[[Variables]]
    B0:   9.3389e-10 +/- 2.25e-12 (0.24%) (init= 1e-09)
    f:    1.2884e+13 +/- 3.57e+10 (0.28%) (init= 10000)
    k:    3.9707e+05 +/- 3.27e+03 (0.82%) (init= 1)
[[Correlations]] (unreported correlations are <  0.100)
    C(B0, f)                     = -0.714 
    C(B0, k)                     =  0.585 
    C(f, k)                      =  0.113 
Reactive ssDNA: 92.0%
F0: 12032 -> Finf: 3700
Fmax: 13064
kf: 3.97e+05
ke: -1.14e-04
t12: 2351.4
Parameters([('B0', <Parameter 'B0', 1e-09, bounds=[-inf:inf]>), ('f', <Parameter 'f', 10000, bounds=[-inf:inf]>), ('k', <Parameter 

In [5]:
#The reaction half times in absence of Redβ are 541 s for the restricted fit, and 396 s for the free fit.
i = 1

A0 = 0.66/199*200/200*195*1e-9  # total volume 199 instead of 200 µl, only taken 195 µl instead of 200 µl: 0.65 nM
B0i = 1/394*400*1e-9  # total volume 394 instead of 400 µl: 1.02 nM
B0f = outs[i].params['B0'].value #* B0
kf = outs[i].params['k'].value
f = outs[i].params['f']

a_ = f * B0f / A0 * (A0 - B0f)
b_ = B0f / A0
ke_ = (A0 - B0f) * kf

##F0 = residual(outs[i].params, model_func, 0)
##Finf = residual(outs[i].params, model_func, np.inf)
F0 = a_ / (1 - b_)
Finf = - a_ / b_

B0e = B0i / b_
k = ke_ / (A0 - B0e)
t12 = np.log(b_ / (2 * b_ - 1)) / ke_

print('Reactive ssDNA: {:.1%}'.format(B0e/B0i))
print('F0: {:.0f}'.format(F0))
print('Finf: {:.0f}'.format(Finf))
print('kf: {:.2e}'.format(kf))
print('ke: {:.2e}'.format(ke_))
print('k: {:.0f}'.format(k))
print('t12: {:.1f}'.format(t12))

Reactive ssDNA: 84.5%
F0: 16633
Finf: 2573
kf: 1.86e+05
ke: -2.20e-05
k: 104235
t12: 6527.5


In [6]:
# Get the data to be fitted
index = 1
dataset = datasets[index]

min_x = 1000
max_x = 40000
x = dataset['data'][:,0]
data = dataset['data'][:,1]
idx = np.logical_and(x >= min_x, x <= max_x)
x = x[idx]
data = data[idx]


# define the model function describing the data
# The first argument needs to be the independent variable (e.g. time)
# The following arguments are the dependent variables. A default value will be used as a start value for the fitting. 
def exponential(t, A=10000, K=-0.001, C=0):
    return A * np.exp(K * t) + C
model_func = exponential


# Fit the data
params = create_params(model_func)#, defaults=(16000, -0.0005, 2000))
out = minimize(residual, params, args=(model_func, x, data))
print(fit_report(out))


# Plot the data and the fit
plt.close('all')
with cnps.cn_plot(context='notebook', dark=False, right_spine=True):
    fig = plt.figure()
    ax = host_subplot(111)
    lns1 = ax.plot(x, data, label='data')
    cnps.set_axis_color()

    ax2 = ax.twinx()
    lns2 = ax2.plot(x, residual(out.params, model_func, x), label='fit')
    cnps.set_axis_color(ax=ax2)

    cnps.legend(lns1, lns2)

    ax.set_title('Example fit of an exponentail decay')
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Counts')
    ax2.set_ylabel('Counts')
    
    fig.show()

Parameters([('A', <Parameter 'A', 10000, bounds=[-inf:inf]>), ('K', <Parameter 'K', -0.001, bounds=[-inf:inf]>), ('C', <Parameter 'C', 0, bounds=[-inf:inf]>)])
[[Fit Statistics]]
    # function evals   = 36
    # data points      = 3901
    # variables        = 3
    chi-square         = 84336833.289
    reduced chi-square = 21635.924
    Akaike info crit   = 38943.211
    Bayesian info crit = 38962.018
[[Variables]]
    A:   11761.2871 +/- 11.68462 (0.10%) (init= 10000)
    K:  -0.00010331 +/- 2.36e-07 (0.23%) (init=-0.001)
    C:   4000.12059 +/- 5.847643 (0.15%) (init= 0)
[[Correlations]] (unreported correlations are <  0.100)
    C(K, C)                      = -0.830 
    C(A, K)                      = -0.477 


FigureCanvasNbAgg()

In [4]:
# see https://stackoverflow.com/questions/3938042/fitting-exponential-decay-with-no-initial-guessing#3938548
def main():
    # Actual parameters
    A0, K0, C0 = 2.5, -4.0, 2.0

    # Generate some data based on these
    tmin, tmax = 0, 0.5
    num = 20
    t = np.linspace(tmin, tmax, num)
    y = model_func(t, A0, K0, C0)

    # Add noise
    noisy_y = y + 0.5 * (np.random.random(num) - 0.5)

    fig = plt.figure()
    ax1 = fig.add_subplot(2,1,1)
    ax2 = fig.add_subplot(2,1,2)

    # Non-linear Fit
    A, K, C = fit_exp_nonlinear(t, noisy_y)
    fit_y = model_func(t, A, K, C)
    plot(ax1, t, y, noisy_y, fit_y, (A0, K0, C0), (A, K, C0))
    ax1.set_title('Non-linear Fit')

    # Linear Fit (Note that we have to provide the y-offset ("C") value!!
    A, K = fit_exp_linear(t, y, C0)
    fit_y = model_func(t, A, K, C0)
    plot(ax2, t, y, noisy_y, fit_y, (A0, K0, C0), (A, K, 0))
    ax2.set_title('Linear Fit')

    plt.show()

def fit_exp_linear(t, y, C=0):
    y = y - C
    y = np.log(y)
    K, A_log = np.polyfit(t, y, 1)
    A = np.exp(A_log)
    return A, K

def model_func(t, A, K, C):
    return A * np.exp(K * t) + C

def fit_exp_nonlinear(t, y):
    opt_parms, parm_cov = sp.optimize.curve_fit(model_func, t, y, maxfev=100000)
    parm_err = np.sqrt(np.diag(parm_cov))
    A, K, C = opt_parms
    return A, K, C

def plot(ax, t, y, noisy_y, fit_y, orig_parms, fit_parms):
    A0, K0, C0 = orig_parms
    A, K, C = fit_parms

    ax.plot(t, y, 'k--', 
      label='Actual Function:\n $y = %0.2f e^{%0.2f t} + %0.2f$' % (A0, K0, C0))
    ax.plot(t, fit_y, 'b-',
      label='Fitted Function:\n $y = %0.2f e^{%0.2f t} + %0.2f$' % (A, K, C))
    ax.plot(t, noisy_y, 'ro')
    ax.legend(bbox_to_anchor=(1.05, 1.1), fancybox=True, shadow=True)